In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import cactas as C
import tensorflow as tf
import pickle

2024-06-19 15:21:27.620047: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
DATAPATH='/raid/mpsych/CACTAS/DATA/ESUS'
CAPATH='/raid/mpsych/CACTAS/DATA/CA24'

In [4]:
images, labels = C.Helper.load_data(DATAPATH)

In [5]:
masks = C.Helper.load_seg_data(CAPATH)

In [6]:
X_train, y_train, m_train, X_test, y_test, m_test = C.Helper.split_patients_masks(images, labels, masks)

In [7]:
with open('/raid/mpsych/CACTAS/DATA/HISTORY/unet_30/train_patient_order.pkl', 'wb') as f:
    pickle.dump(X_train, f)
with open('/raid/mpsych/CACTAS/DATA/HISTORY/unet_30/test_patient_order.pkl', 'wb') as f:
    pickle.dump(X_test, f)

In [8]:
X_train, y_train, m_train, X_test, y_test, m_test = C.Helper.normalization2(DATAPATH, CAPATH, X_train, y_train, m_train, X_test, y_test, m_test)

In [9]:
slice_to_patient_mapping, last_key = C.Helper.map_and_key_fulldata(y_train)

10679


In [10]:
X_train, y_train, X_test, y_test = C.Helper.extract_slices(X_train, y_train, X_test, y_test)

(13157, 512, 512, 1) (13157, 512, 512, 1) (3609, 512, 512, 1) (3609, 512, 512, 1)


In [11]:
m_train, m_test = C.Helper.extract_masks_slices(m_train, m_test)

(13157, 512, 512, 1) (3609, 512, 512, 1)


In [12]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
m_train = m_train.astype(np.float32)
m_test = m_test.astype(np.float32)

In [13]:
train_images_array, test_images_array = C.Helper.masked_image(X_train, m_train, X_test, m_test)

(13157, 512, 512, 1) (3609, 512, 512, 1)


In [14]:
from sklearn.model_selection import KFold

In [23]:
n_samples = len(train_images_array)
kf = KFold(n_splits=n_samples // 1000)
#kf = KFold(n_splits=2, shuffle=False)
histories = []
fold = 1

In [24]:
kf

KFold(n_splits=13, random_state=None, shuffle=False)

In [ ]:
# 5 = 2631 

In [ ]:
#10 = 1315

In [ ]:
# 50 = 263

In [ ]:
# 100 = 131

In [ ]:
# 1000 = 13

In [25]:
losses = []
ious = []
iou_thresholds = []

In [ ]:
for train_index, test_index in kf.split(train_images_array):
    X_train, X_val = train_images_array[train_index], train_images_array[test_index]
    Y_train, Y_val = y_train[train_index], y_train[test_index]

    model = C.Helper.create_unet(train_images_array[0].shape)

    print(f"Training fold {fold}...")
    history = model.fit(X_train, Y_train, 
                        epochs=100, 
                        batch_size=16, 
                        steps_per_epoch=len(X_train) // 16, 
                        validation_data=(X_val, Y_val))
    histories.append(history.history)
    
    scores = model.evaluate(test_images_array, y_test, verbose=0)
    losses.append(scores[0])
    ious.append(scores[1])
    iou_thresholds.append(scores[2])    
    
    fold += 1

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


2024-06-19 16:45:26.084020: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-06-19 16:45:26.217682: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2024-06-19 16:45:26.220749: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:4e:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2024-06-19 16:45:26.220777: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-06-19 16:45:26.236527: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-06-19 

Training fold 1...


2024-06-19 16:45:26.255433: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 16:45:26.455261: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2024-06-19 16:45:26.457798: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:4e:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2024-06-19 16:45:26.467740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible 

Epoch 1/100


2024-06-19 16:45:30.241554: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2024-06-19 16:45:30.951967: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2024-06-19 16:45:31.725375: W tensorflow/stream_executor/gpu/asm_compiler.cc:191] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2024-06-19 16:45:31.725401: W tensorflow/stream_executor/gpu/asm_compiler.cc:194] Used ptxas at ptxas
2024-06-19 16:45:31.725459: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-06-19 16:45:31.800812: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-06-19 16:45:32.616866: I tensorflow/stream_exec

759/759 [==============================] - 330s 419ms/step - loss: 0.0816 - iou: 4.6148e-05 - iou_thresholded: 0.2715 - val_loss: 0.0025 - val_iou: 1.1188e-04 - val_iou_thresholded: 0.8917
Epoch 2/100
759/759 [==============================] - 316s 416ms/step - loss: 0.0012 - iou: 3.3297e-04 - iou_thresholded: 0.4208 - val_loss: 5.8107e-04 - val_iou: 7.8017e-04 - val_iou_thresholded: 0.7939
Epoch 3/100
759/759 [==============================] - 316s 416ms/step - loss: 4.4670e-04 - iou: 0.0011 - iou_thresholded: 0.3825 - val_loss: 2.7322e-04 - val_iou: 0.0018 - val_iou_thresholded: 0.7012
Epoch 4/100
759/759 [==============================] - 316s 416ms/step - loss: 2.1782e-04 - iou: 0.0025 - iou_thresholded: 0.3916 - val_loss: 1.4868e-04 - val_iou: 0.0024 - val_iou_thresholded: 0.8917
Epoch 5/100
759/759 [==============================] - 316s 416ms/step - loss: 1.3967e-04 - iou: 0.0047 - iou_thresholded: 0.3901 - val_loss: 9.4722e-05 - val_iou: 0.0044 - val_iou_thresholded: 0.8553
Epo

Epoch 42/100
759/759 [==============================] - 316s 416ms/step - loss: 1.3717e-05 - iou: 0.3136 - iou_thresholded: 0.6756 - val_loss: 9.4924e-06 - val_iou: 0.5325 - val_iou_thresholded: 0.9252
Epoch 43/100
759/759 [==============================] - 316s 416ms/step - loss: 1.3703e-05 - iou: 0.3232 - iou_thresholded: 0.6648 - val_loss: 1.5668e-05 - val_iou: 0.5901 - val_iou_thresholded: 0.8980
Epoch 44/100
759/759 [==============================] - 315s 416ms/step - loss: 1.2900e-05 - iou: 0.3285 - iou_thresholded: 0.6706 - val_loss: 1.1693e-05 - val_iou: 0.5368 - val_iou_thresholded: 0.9256
Epoch 45/100
759/759 [==============================] - 315s 415ms/step - loss: 1.2287e-05 - iou: 0.3583 - iou_thresholded: 0.6842 - val_loss: 1.6710e-05 - val_iou: 0.2925 - val_iou_thresholded: 0.9105
Epoch 46/100
759/759 [==============================] - 315s 415ms/step - loss: 1.2071e-05 - iou: 0.3562 - iou_thresholded: 0.6826 - val_loss: 1.2339e-05 - val_iou: 0.6797 - val_iou_thresholde

759/759 [==============================] - 324s 427ms/step - loss: 2.5966e-05 - iou: 0.1529 - iou_thresholded: 0.5104 - val_loss: 2.6503e-05 - val_iou: 0.2426 - val_iou_thresholded: 0.8895
Epoch 23/100
759/759 [==============================] - 324s 427ms/step - loss: 2.6002e-05 - iou: 0.1512 - iou_thresholded: 0.4938 - val_loss: 2.4120e-05 - val_iou: 0.3335 - val_iou_thresholded: 0.8833
Epoch 24/100
759/759 [==============================] - 324s 426ms/step - loss: 2.4630e-05 - iou: 0.1688 - iou_thresholded: 0.5214 - val_loss: 3.7052e-05 - val_iou: 0.2695 - val_iou_thresholded: 0.8921
Epoch 25/100
759/759 [==============================] - 324s 426ms/step - loss: 2.3062e-05 - iou: 0.1852 - iou_thresholded: 0.5259 - val_loss: 3.0353e-05 - val_iou: 0.3938 - val_iou_thresholded: 0.8498
Epoch 26/100
759/759 [==============================] - 324s 426ms/step - loss: 2.2096e-05 - iou: 0.1877 - iou_thresholded: 0.5578 - val_loss: 3.3940e-05 - val_iou: 0.3061 - val_iou_thresholded: 0.8768
Epo

Epoch 63/100
759/759 [==============================] - 324s 426ms/step - loss: 7.6508e-06 - iou: 0.5253 - iou_thresholded: 0.7878 - val_loss: 4.2913e-05 - val_iou: 0.6225 - val_iou_thresholded: 0.8421
Epoch 64/100
759/759 [==============================] - 324s 427ms/step - loss: 7.2427e-06 - iou: 0.5511 - iou_thresholded: 0.8088 - val_loss: 5.1136e-05 - val_iou: 0.6312 - val_iou_thresholded: 0.8802
Epoch 65/100
759/759 [==============================] - 324s 426ms/step - loss: 7.2741e-06 - iou: 0.5561 - iou_thresholded: 0.7948 - val_loss: 3.8204e-05 - val_iou: 0.6851 - val_iou_thresholded: 0.8869
Epoch 66/100
759/759 [==============================] - 324s 427ms/step - loss: 7.2264e-06 - iou: 0.5523 - iou_thresholded: 0.8006 - val_loss: 3.9091e-05 - val_iou: 0.6645 - val_iou_thresholded: 0.8940
Epoch 67/100
759/759 [==============================] - 324s 426ms/step - loss: 7.9912e-06 - iou: 0.5343 - iou_thresholded: 0.8036 - val_loss: 4.8245e-05 - val_iou: 0.6308 - val_iou_thresholde